In [1]:
import sympy as sp
# from sympy import *

# ----- Symbols -----
alpha, beta, psi = sp.symbols('alpha beta psi', real=True)   # GW direction & pol angle
omega = sp.symbols('omega', positive=True, real=True)        # GW angular frequency
t = sp.symbols('t', real=True)

# Strain amplitudes (time-dependent, keep symbolic)
h_plus  = sp.symbols('h_+')
h_cross = sp.symbols('h_\\times')

# helicity basis
h_p2, h_m2 = sp.symbols('h_+2 h_-2')

# Lab/PD spatial coordinates and vectors
x, y, z = sp.symbols('x y z', real=True)
r, phi = sp.symbols('r phi', real=True)  # cylindrical coords
X = sp.Matrix([x, y, z])

# ----- GW direction and transverse basis (in lab/PD axes) -----
# nx = sp.sin(beta)*sp.cos(alpha)
# ny = sp.sin(beta)*sp.sin(alpha)
# nz = sp.cos(beta)

nx, ny, nz = sp.symbols('nx ny nz', real=True)
n = sp.Matrix([nx, ny, nz])

# # Spherical tangent basis at n̂
# th = sp.Matrix([sp.cos(beta)*sp.cos(alpha), sp.cos(beta)*sp.sin(alpha), -sp.sin(beta)])
# ph = sp.Matrix([-sp.sin(alpha),             sp.cos(alpha),              0])

# # Rotate by polarization angle ψ about n̂ to get (p,q)
# p =  th*sp.cos(psi) + ph*sp.sin(psi)
# q = -th*sp.sin(psi) + ph*sp.cos(psi)

# # ----- Polarization tensors in lab/PD basis -----
# e_plus  = sp.Matrix([[p[i]*p[j] - q[i]*q[j] for j in range(3)] for i in range(3)])
# e_cross = sp.Matrix([[p[i]*q[j] + q[i]*p[j] for j in range(3)] for i in range(3)])

exx_p, exy_p, exz_p, eyy_p, eyz_p, ezz_p = sp.symbols('e_xx^+ e_xy^+ e_xz^+ e_yy^+ e_yz^+ e_zz^+', real=True)
exx_c, exy_c, exz_c, eyy_c, eyz_c, ezz_c = sp.symbols('e_xx^\\times e_xy^\\times e_xz^\\times' \
'                                                    e_yy^\\times e_yz^\\times e_zz^\\times', real=True)

e_plus = sp.Matrix([[exx_p, exy_p, exz_p],
                   [exy_p, eyy_p, eyz_p],
                  [exz_p, eyz_p, ezz_p]])

e_cross = sp.Matrix([[exx_c, exy_c, exz_c],
                    [exy_c, eyy_c, eyz_c],
                    [exz_c, eyz_c, ezz_c]])

# Combine pols with (symbolic) strains
hTT = h_plus*e_plus + h_cross*e_cross   # S_ij ≡ Σ_A h_A e^A_ij
berlin = {alpha:0, beta:0, psi:0,
          nx:0, ny:0, nz:1,
          exx_p:1,eyy_p:-1,ezz_p:0,exy_p:0,exz_p:0,eyz_p:0,
          exx_c:0,eyy_c:0,ezz_c:0,exy_c:1,exz_c:0,eyz_c:0} # hTT_ij(i,j).subs(berlin) gives on axis GW result in Berlin


cart2cyl = {x:r*sp.cos(phi), y:r*sp.sin(phi), z:z}



Q = sp.symbols('Q', positive=True)
F0, F1, F2 = sp.symbols('F0 F1 F2', real=True)
F0p, F1p, F2p = sp.symbols('F0p F1p F2p', real=True)

# helpers
ndotx = Q/omega
hx    = hTT*X                       # (S·x)_i
nhx   = n.dot(hx)                  # n · (S·x)
S = hx.dot(X)

def hTT_ij(i, j):
    return hTT[i-1, j-1]
def hTT_ijxj(i):
    return hx[i-1]

def V_i(i):
    ni = n[i-1]
    return ndotx*hTT_ijxj(i) - ni*S
def T_ij(i, j):
    ni = n[i-1]
    nj = n[j-1]
    return ndotx*nj*hTT_ijxj(i)+ndotx*ni*hTT_ijxj(j)-ni*nj*S-(ndotx**2)*hTT_ij(i, j)

Tii = T_ij(1,1) + T_ij(2,2) + T_ij(3,3)

def ni_hTT_ij(j):
    return n[0]*hTT_ij(1, j) + n[1]*hTT_ij(2, j) + n[2]*hTT_ij(3, j)

def dmTii(m):
    nm = n[m-1]
    return 2*omega**2*(ndotx*ni_hTT_ij(m)-hTT_ijxj(m)+nm*nhx)

# partial derivatives of PD frame h_uv components and trace

def d0h_0i(i):
    ni = n[i-1]
    return -1.0j*omega*(-omega**2*F1*(ndotx*hTT_ijxj(i)-ni*S))

def dmh_00(m):
    return -2*omega**2*hTT_ijxj(m)*F0-omega**2*S*F0p*n[m-1]*omega

def dmh_0i(m, i):
    nm = n[m-1]
    ni = n[i-1]
    return -omega**2*F1*(ndotx*hTT_ij(m,i)+nm*hTT_ijxj(i)-2*ni*hTT_ijxj(m))-omega**2*V_i(i)*F1p*nm*omega

def dmh_ij(m, i, j):
    nm = n[m-1]
    ni = n[i-1]
    nj = n[j-1]
    return omega**2*F2*((nm*nj*hTT_ijxj(i)+nm*ni*hTT_ijxj(j)-2*ni*nj*hTT_ijxj(m)) + ndotx*(ni*hTT_ij(j,m)+nj*hTT_ij(i,m)-2*nm*hTT_ij(i,j))) + omega**2*T_ij(i, j)*F2p*nm*omega

def dmh_trace(m):
    # return -dmh_00(m) + dmh_ij(m,1,1) + dmh_ij(m,2,2) + dmh_ij(m,3,3)
    return 0

Symbolic J0

In [2]:
j0_over_Bw2 = ((dmh_0i(1,2)-dmh_0i(2,1))/omega**2).expand().subs(cart2cyl).collect([h_plus, h_cross])
j0_p = j0_over_Bw2.coeff(h_plus).collect([F1*Q, 3*F1, F1p*Q])
j0_c = j0_over_Bw2.coeff(h_cross).collect([F1*Q, 3*F1, F1p*Q])

In [3]:
j0_p = j0_p.simplify()
j0_c = j0_c.simplify()

j0_p

(3*F1 + F1p*Q)*(e_xx^+*ny*r*cos(phi) - e_xy^+*nx*r*cos(phi) + e_xy^+*ny*r*sin(phi) + e_xz^+*ny*z - e_yy^+*nx*r*sin(phi) - e_yz^+*nx*z)

Symbolic Jx

In [4]:
jx_over_Bw2 = ((-0.5*dmh_trace(2)-d0h_0i(2)+dmh_ij(2,2,2)+dmh_ij(3,3,2)+dmh_ij(2,1,1))/omega**2)         \
                .subs(cart2cyl).expand().collect([h_plus, h_cross])
# jx_over_Bw2 = ((dmh_ij(1,2,1)+d0h_0i(2)-dmh_ij(3,3,2)-dmh_ij(2,1,1))/omega**2).subs(cart2cyl).expand().collect([h_plus, h_cross])
jx_p = jx_over_Bw2.coeff(h_plus).collect([F1*Q, F1, F2, F2*Q, F2p, F2p*Q, F2p*Q*Q])
jx_c = jx_over_Bw2.coeff(h_cross).collect([F1*Q, F1, F2, F2*Q, F2p, F2p*Q, F2p*Q*Q])

Symbolic Jy

In [5]:
jy_over_Bw2 = ((0.5*dmh_trace(1)+d0h_0i(1)-dmh_ij(1,1,1)-dmh_ij(3,3,1)-dmh_ij(1,2,2))/omega**2)    \
    .subs(cart2cyl).expand().collect([h_plus, h_cross])
# jy_over_Bw2 = ((2*dmh_ij(1,1,1)-d0h_0i(1)+dmh_ij(2,2,1)+dmh_ij(3,3,1)
#              +dmh_ij(1,2,2))/omega**2).subs(cart2cyl).expand().collect([h_plus, h_cross])
jy_p = jy_over_Bw2.coeff(h_plus).collect([F1*Q, F1, F2, F2*Q, F2p, F2p*Q, F2p*Q*Q])
jy_c = jy_over_Bw2.coeff(h_cross).collect([F1*Q, F1, F2, F2*Q, F2p, F2p*Q, F2p*Q*Q])

(Jx, Jy) -> (Jr, Jphi)

In [6]:
jr_over_Bw2 = jx_over_Bw2*sp.cos(phi) + jy_over_Bw2*sp.sin(phi)
jphi_over_Bw2 = -jx_over_Bw2*sp.sin(phi) + jy_over_Bw2*sp.cos(phi)

jr_p = jr_over_Bw2.expand().coeff(h_plus).collect([F1*Q,F1,F2*Q,F2,F2p,F2p*Q,F2p*Q*Q]).collect([exx_p, 
            exy_p, exz_p, eyy_p, eyz_p, ezz_p])
jr_c = jr_over_Bw2.expand().coeff(h_cross).collect([F1*Q,F1,F2*Q,F2,F2p,F2p*Q,F2p*Q*Q]).collect([exx_c, 
            exy_c, exz_c, eyy_c, eyz_c, ezz_c])

jphi_p = jphi_over_Bw2.expand().coeff(h_plus).collect([F1*Q,F1,F2*Q,F2,F2p,F2p*Q,F2p*Q*Q]).collect([exx_p, 
            exy_p, exz_p, eyy_p, eyz_p, ezz_p])
jphi_c = jphi_over_Bw2.expand().coeff(h_cross).collect([F1*Q,F1,F2*Q,F2,F2p,F2p*Q,F2p*Q*Q]).collect([exx_c, 
            exy_c, exz_c, eyy_c, eyz_c, ezz_c])

In [7]:
jr_p = jr_p.expand().collect([exx_p, eyy_p, exy_p, exz_p, eyz_p, ezz_p])
jr_c = jr_c.expand().collect([exx_c, eyy_c, exy_c, exz_c, eyz_c, ezz_c])
jphi_p = jphi_p.expand().collect([exx_p, eyy_p, exy_p, exz_p, eyz_p, ezz_p])
jphi_c = jphi_c.expand().collect([exx_c, eyy_c, exy_c, exz_c, eyz_c, ezz_c])

jr_p

e_xx^+*(1.0*I*F1*Q*r*sin(phi)*cos(phi) - 1.0*I*F1*nx*omega*r**2*sin(phi)*cos(phi)**2 + 1.0*I*F1*ny*omega*r**2*cos(phi)**3 - 2*F2*Q*ny*cos(phi)/omega + 2*F2*nx*ny*r*cos(phi)**2 + 2*F2*ny**2*r*sin(phi)*cos(phi) - F2*nz**2*r*sin(phi)*cos(phi) + F2p*Q**2*nx*sin(phi)/omega - F2p*Q**2*ny*cos(phi)/omega - 2*F2p*Q*nx**2*r*sin(phi)*cos(phi) + 2*F2p*Q*nx*ny*r*cos(phi)**2 - F2p*Q*nz**2*r*sin(phi)*cos(phi) + F2p*nx**3*omega*r**2*sin(phi)*cos(phi)**2 - F2p*nx**2*ny*omega*r**2*cos(phi)**3 + F2p*nx*ny**2*omega*r**2*sin(phi)*cos(phi)**2 + F2p*nx*nz**2*omega*r**2*sin(phi)*cos(phi)**2 - F2p*ny**3*omega*r**2*cos(phi)**3 - F2p*ny*nz**2*omega*r**2*cos(phi)**3) + e_xy^+*(1.0*I*F1*Q*r*sin(phi)**2 - 1.0*I*F1*Q*r*cos(phi)**2 - 2.0*I*F1*nx*omega*r**2*sin(phi)**2*cos(phi) + 2.0*I*F1*ny*omega*r**2*sin(phi)*cos(phi)**2 + 2*F2*Q*nx*cos(phi)/omega - 2*F2*Q*ny*sin(phi)/omega - 2*F2*nx**2*r*cos(phi)**2 + 2*F2*ny**2*r*sin(phi)**2 - F2*nz**2*r*sin(phi)**2 + F2*nz**2*r*cos(phi)**2 - 2*F2p*Q*nx**2*r*sin(phi)**2 + 2*F2p*Q*

In [8]:
jr_over_Bw2.expand().subs(berlin).trigsimp().collect([h_plus, h_cross])

1.0*r*(h_+*(I*F1*Q*sin(2*phi) - F2*sin(2*phi) - F2p*Q*sin(2*phi)) + h_\times*(-I*F1*Q*cos(2*phi) + F2*cos(2*phi) + F2p*Q*cos(2*phi)))

Helicity basis (Berlin eq 24):

In [9]:
write_in_hel_basis = {h_plus:(h_p2+h_m2)/sp.sqrt(2), h_cross:-1j*(h_p2-h_m2)/sp.sqrt(2)}
jr_over_Bw2.subs(write_in_hel_basis).expand().rewrite(sp.exp).expand().powsimp().collect([h_p2, h_m2]).subs(berlin)

h_+2*(-0.5*sqrt(2)*F1*Q*r*exp(-2*I*phi) - 0.5*sqrt(2)*I*F2*r*exp(-2*I*phi) - 0.5*sqrt(2)*I*F2p*Q*r*exp(-2*I*phi)) + h_-2*(0.5*sqrt(2)*F1*Q*r*exp(2*I*phi) + 0.5*sqrt(2)*I*F2*r*exp(2*I*phi) + 0.5*sqrt(2)*I*F2p*Q*r*exp(2*I*phi))

In [10]:
jphi_over_Bw2.subs(write_in_hel_basis).expand().rewrite(sp.exp).expand().powsimp().collect([h_p2, h_m2]).subs(berlin)

h_+2*(0.5*sqrt(2)*I*F1*Q*r*exp(-2*I*phi) - 0.5*sqrt(2)*F2*r*exp(-2*I*phi) - 0.5*sqrt(2)*F2p*Q*r*exp(-2*I*phi)) + h_-2*(0.5*sqrt(2)*I*F1*Q*r*exp(2*I*phi) - 0.5*sqrt(2)*F2*r*exp(2*I*phi) - 0.5*sqrt(2)*F2p*Q*r*exp(2*I*phi))

The F-factors should have matched Berlin eq 25:

In [11]:
f0_Q = (-1j/Q + (1-sp.exp(-1j*Q))/Q**2)
f1_Q = (-1j/(2*Q) - sp.exp(-1j*Q)/Q**2 - 1j*(1-sp.exp(-1j*Q))/Q**3)
f2_Q = (-(1+sp.exp(-1j*Q))/Q**2 - 2j*(1-sp.exp(-1j*Q))/Q**3)

f0p_Q = sp.diff(f0_Q, Q)
f1p_Q = sp.diff(f1_Q, Q)
f2p_Q = sp.diff(f2_Q, Q)

(3j*(Q*f1_Q + 1j*f2_Q + 1j*f2p_Q*Q)).expand().powsimp()

1.5 - 6.0*I*exp(-1.0*I*Q)/Q - 12.0*exp(-1.0*I*Q)/Q**2 - 12.0*I/Q**3 + 12.0*I*exp(-1.0*I*Q)/Q**3

My first term is +1.5, while Berlin has -3. The rest are the same.

Symbolic Jz

In [12]:
jz_over_Bw2 = ((dmh_ij(2,3,1)-dmh_ij(1,3,2))/omega**2).subs(cart2cyl).expand().collect([h_plus, h_cross])
jz_p = jz_over_Bw2.coeff(h_plus).collect([F2*Q, F2, F2p*Q])
jz_c = jz_over_Bw2.coeff(h_cross).collect([F2*Q, F2, F2p*Q])

In [13]:
jz_p = jz_p.expand().collect([exx_p, eyy_p, exy_p, exz_p, eyz_p, ezz_p])
jz_c = jz_c.expand().collect([exx_c, eyy_c, exy_c, exz_c, eyz_c, ezz_c])

jz_p

e_xx^+*(3*F2*ny*nz*r*cos(phi) + F2p*Q*ny*nz*r*cos(phi)) + e_xy^+*(-3*F2*nx*nz*r*cos(phi) + 3*F2*ny*nz*r*sin(phi) - F2p*Q*nx*nz*r*cos(phi) + F2p*Q*ny*nz*r*sin(phi)) + e_xz^+*(-3*F2*Q*ny/omega + 3*F2*ny*nz*z - F2p*Q**2*ny/omega + F2p*Q*ny*nz*z) + e_yy^+*(-3*F2*nx*nz*r*sin(phi) - F2p*Q*nx*nz*r*sin(phi)) + e_yz^+*(3*F2*Q*nx/omega - 3*F2*nx*nz*z + F2p*Q**2*nx/omega - F2p*Q*nx*nz*z)

In [ ]:
e_syms_p = [exx_p, exy_p, exz_p, eyy_p, eyz_p, ezz_p]
e_syms_c = [exx_c, exy_c, exz_c, eyy_c, eyz_c, ezz_c]
F_monos = [F1, F1*Q, F2, F2*Q, F2p, F2p*Q, F2p*Q**2]

def extract_exprs(expr, e_syms, F_monos):
    expr = sp.expand(expr)
    out = {}
    for j, eij in enumerate(e_syms):
        part = sp.expand(sp.collect(expr, eij).coeff(eij))
        for k, mono in enumerate(F_monos):
            c = sp.expand(sp.collect(part, mono).coeff(mono))
            if c != 0:
                out[(j,k)] = c
    return out

JR_plus_exprs   = extract_exprs(jr_p,   e_syms_p, F_monos)
JPHI_plus_exprs = extract_exprs(jphi_p, e_syms_p, F_monos)
JZ_plus_exprs   = extract_exprs(jz_p,   e_syms_p, F_monos)

JR_cross_exprs   = extract_exprs(jr_c,   e_syms_c, F_monos)
JPHI_cross_exprs = extract_exprs(jphi_c, e_syms_c, F_monos)
JZ_cross_exprs   = extract_exprs(jz_c,   e_syms_c, F_monos)

import pickle, sympy as sp, numpy as np
bundle = {
    "sympy_version": sp.__version__,
    "numpy_version": np.__version__,
    "vars": ("r","phi","z","nx","ny","nz","Q","omega"),
    "F_monos": ["F1","Q*F1","F2","Q*F2","F2p","Q*F2p","Q**2*F2p"],
    "JR_plus_exprs":   JR_plus_exprs,
    "JPHI_plus_exprs": JPHI_plus_exprs,
    "JZ_plus_exprs":   JZ_plus_exprs,
    "JR_cross_exprs":   JR_cross_exprs,
    "JPHI_cross_exprs": JPHI_cross_exprs,
    "JZ_cross_exprs":   JZ_cross_exprs
}
with open("/grad/sguotong/projects/CaGe/data/j_eff_expr/j_eff_kernels.pkl","wb") as f:
    pickle.dump(bundle, f)